## 5. Hydrogen Atom: Variational Method

The variational method requires taking a system and making an educated guess for that system. Let's think about the hydrogen atom. It has an electron and a proton. Experimentally, we know that the atom has some maximum radius, which we will call $a_0$. The electron then must have its wavefunction go to zero outside of the radius of the atom. Otherwise, we might find an electron very far away from the atom. Of course, strictly speaking experimentally, we would like to not completely rule out this possibility, so let's say it's very small or close to zero outside of $a_0$. What about as the electron approaches the proton, i.e. $r \rightarrow 0$? The probability has to add up to 1, so it cannot go to infinity. From Rutherford's gold foil experiment, we know that the atom is comprised of a small, dense nucleus, with electrons some distance from the nucleus. So, the electrons are unlikely to be at $r=0$. Thus, we assume that the probability goes to zero as $r \rightarrow 0$.

Since we are talking about the probability here, we are looking for a function $\psi(r, \theta, \phi)$ such that:

$$
f(r, \theta, \phi) = \left|\psi(r, \theta, \phi)\right|^2 r^2
$$

has the aforementioned properties.

One guess is:

$$
u(r) = r \theta_0 e^{-(r-\theta_1)/\theta_2}
$$